<a href="https://www.kaggle.com/code/ayobamimike/medical-imagesdenoisingusingnoise2void?scriptVersionId=223025649" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## This is a PyTorch implementation of a Denoising Autoencoder (DAE) to remove noise from dental X-ray images. The model is trained in an unsupervised manner, meaning it does not require paired clean/noisy images. 

#### The Denoising Autoencoder (DAE) is a neural network-based approach designed to remove noise from dental X-ray images. It is built on a U-Net architecture, which captures fine details and reconstructs clean images from noisy inputs.

#### This implementation processes 120 dental images, extracting useful features while eliminating unwanted noise.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import cv2
import os


In [2]:
class NoisyImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_filenames = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        img = img / 255.0  # Normalize
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel dim

        return img, img  # Input = Noisy, Target = Itself (Autoencoder)


In [3]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 1, kernel_size=3, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = nn.functional.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
        x = self.decoder(x)
        return x


In [4]:
# Load dataset
dataset = NoisyImageDataset("/kaggle/input/medical-image-dataset/Dataset")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for noisy, target in dataloader:
        noisy, target = noisy.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(noisy)
        loss = criterion(output, target)  # Minimize noise
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

# Save model
torch.save(model.state_dict(), "denoising_autoencoder.pth")


Epoch 1/10, Loss: 0.0130
Epoch 2/10, Loss: 0.0011
Epoch 3/10, Loss: 0.0006
Epoch 4/10, Loss: 0.0004
Epoch 5/10, Loss: 0.0003
Epoch 6/10, Loss: 0.0003
Epoch 7/10, Loss: 0.0003
Epoch 8/10, Loss: 0.0002
Epoch 9/10, Loss: 0.0002
Epoch 10/10, Loss: 0.0002


In [5]:
def denoise_image(model, image_path):
    model.eval()
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) / 255.0
    img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        denoised = model(img_tensor).squeeze().cpu().numpy()

    return (denoised * 255).astype(np.uint8)

# Load trained model
model.load_state_dict(torch.load("denoising_autoencoder.pth", map_location=device))

# Denoise and save all images
input_folder = "/kaggle/input/medical-image-dataset/Dataset"
output_folder = "path/to/denoised_images"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)

    denoised_img = denoise_image(model, input_path)
    cv2.imwrite(output_path, denoised_img)

print("Denoising complete. Check the output folder!")


<ipython-input-5-9d2e7acf38a2>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("denoising_autoencoder.pth", map_location=device))


Denoising complete. Check the output folder!
